sometimes code is too fast for browser driver. in such cases, try increasing sleep time.

In [1]:
from bs4 import BeautifulSoup as bs 
import math
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import os
 

In [2]:
DRIVER_PATH = 'C:/Users/thisi/Documents/GitHub/pq-scraper/chromedriver_win32/chromedriver.exe'

In [3]:
def element_completely_viewable(driver, elem):
    elem_left_bound = elem.location.get('x')
    elem_top_bound = elem.location.get('y')
    elem_width = elem.size.get('width')
    elem_height = elem.size.get('height')
    elem_right_bound = elem_left_bound + elem_width
    elem_lower_bound = elem_top_bound + elem_height

    win_upper_bound = driver.execute_script('return window.pageYOffset')
    win_left_bound = driver.execute_script('return window.pageXOffset')
    win_width = driver.execute_script('return document.documentElement.clientWidth')
    win_height = driver.execute_script('return document.documentElement.clientHeight')
    win_right_bound = win_left_bound + win_width
    win_lower_bound = win_upper_bound + win_height

    return all((win_left_bound <= elem_left_bound,
                win_right_bound >= elem_right_bound,
                win_upper_bound <= elem_top_bound,
                win_lower_bound >= elem_lower_bound)
              )

def find_and_click(driver, elem):
    y_height = 0
    while not element_completely_viewable(driver, elem):
        driver.execute_script(f"window.scrollTo(0, {y_height})") 
        y_height += 100
        time.sleep(0.5)
    elem.click()

driver = webdriver.Chrome(DRIVER_PATH) 
 
page_url = 'https://www.parliament.gov.sg/history/list-of-mps-by-parliament' 
driver.get(page_url) 
driver.maximize_window() 
driver.implicitly_wait(20) 
time.sleep(2) 

parliament_combobox = driver.find_element(
    by=By.ID,
    value='Parliament'
)

num_parliaments = int(parliament_combobox.get_attribute('childElementCount'))
print(f'Total of {num_parliaments} parliaments')


C:\Users\thisi\AppData\Local\Temp\ipykernel_7632\1172286970.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH)


Total of 14 parliaments


In [4]:
mps_by_parliament = []

for parliament_index in range(1, num_parliaments + 1):
    # need this line of code to prevent stale element reference. not sure why but just keep it. 
    parliament_combobox = driver.find_element(
        by=By.ID,
        value='Parliament'
    )

    current_parliament_option = driver.find_element(
        by=By.XPATH,
        value=f'//*[@id="Parliament"]/option[{parliament_index}]'
    )
    
    find_and_click(driver, parliament_combobox)
    time.sleep(0.5)
    find_and_click(driver, current_parliament_option)
    time.sleep(2)
    
    print(f'Looking at parliament {parliament_index}')
    
    full_table = driver.find_element(
        by=By.XPATH,
        value='/html/body/form/div[3]/div[2]/div[2]/div[1]/div[3]/ul'
    )
    
    num_rows = int(full_table.get_attribute('childElementCount'))
    print(f'Found {num_rows} MPs')
    
    mps_this_parliament = []
    
    for i in range(1, num_rows+1):
        name_div = driver.find_element(
            by=By.XPATH,
            value=f'/html/body/form/div[3]/div[2]/div[2]/div[1]/div[3]/ul/li[{i}]/div/div[1]/div[2]'
        )
        
        party_div = driver.find_element(
            by=By.XPATH,
            value=f'/html/body/form/div[3]/div[2]/div[2]/div[1]/div[3]/ul/li[{i}]/div/div[2]'
        )
        
        legis_assembly_div = driver.find_element(
            by=By.XPATH,
            value=f'/html/body/form/div[3]/div[2]/div[2]/div[1]/div[3]/ul/li[{i}]/div/div[3]'
        )
        
        also_in_div = driver.find_element(
            by=By.XPATH,
            value=f'/html/body/form/div[3]/div[2]/div[2]/div[1]/div[3]/ul/li[{i}]/div/div[4]'
        )
        
        name, party, legis_assembly, also_in = list(map(
            lambda div: div.get_attribute('innerHTML').strip(),
            [name_div, party_div, legis_assembly_div, also_in_div]
        ))
        
        mps_this_parliament.append((name, party, legis_assembly, also_in))
    
    mps_by_parliament.append(mps_this_parliament)

Looking at parliament 1
Found 64 MPs
Looking at parliament 2
Found 64 MPs
Looking at parliament 3
Found 65 MPs
Looking at parliament 4
Found 78 MPs
Looking at parliament 5
Found 76 MPs
Looking at parliament 6
Found 79 MPs
Looking at parliament 7
Found 84 MPs
Looking at parliament 8
Found 92 MPs
Looking at parliament 9
Found 99 MPs
Looking at parliament 10
Found 103 MPs
Looking at parliament 11
Found 103 MPs
Looking at parliament 12
Found 110 MPs
Looking at parliament 13
Found 111 MPs
Looking at parliament 14
Found 102 MPs


In [5]:
mps_by_parliament

[[('A. Rahim Ishak', "People's Action Party", '', '1st, 2nd, 3rd, 4th, 5th'),
  ('A.P. Rajah', 'Independent Singapore Party Alliance', '√', '1st'),
  ('Ang Nam Piau', "People's Action Party", '', '1st, 2nd, 3rd, 4th'),
  ('Bani, S.T.', 'Barisan Sosialis', '√', '1st'),
  ('Barker, E.W.',
   "People's Action Party",
   '',
   '1st, 2nd, 3rd, 4th, 5th, 6th'),
  ('Bernard Rodrigues', "People's Action Party", '', '1st'),
  ('Buang Bin Omar Junid', "People's Action Party", '√', '1st, 2nd'),
  ('Chan Chee Seng', "People's Action Party", '√', '1st, 2nd, 3rd, 4th, 5th'),
  ('Chan Choy Siong', "People's Action Party", '√', '1st, 2nd'),
  ('Chan Sun Wing', "People's Action Party, Barisan Sosialis", '√', '1st'),
  ('Chew Chin Harn', "People's Action Party", '', '1st'),
  ('Chia Thye Poh', 'Barisan Sosialis', '', '1st'),
  ('Chio Cheng Thun', 'Barisan Sosialis', '', '1st'),
  ('Chor Yeok Eng', "People's Action Party", '√', '1st, 2nd, 3rd, 4th, 5th'),
  ('Chow Chiok Hock', "People's Action Party", '